<a href="https://colab.research.google.com/github/Artin200912/Deep-learning/blob/main/Comment_Toxicity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Getting data

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
!mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

In [ ]:
! kaggle competitions download -c 'jigsaw-toxic-comment-classification-challenge'

In [ ]:
!mkdir data

In [ ]:
!unzip /content/jigsaw-toxic-comment-classification-challenge.zip -d data

In [ ]:
!unzip /content/data/sample_submission.csv.zip -d data
!unzip /content/data/test.csv.zip -d data
!unzip /content/data/test_labels.csv.zip -d data
!unzip /content/data/train.csv.zip -d data

# Libraries & data

In [ ]:
!pip list

In [ ]:
!pip install -q gradio jinja2

In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
import matplotlib.pyplot as plt
import gradio as gr

In [ ]:
df = pd.read_csv(os.path.join('/content/data', 'train.csv'))
df.head()

In [ ]:
df.iloc[0]['comment_text']

In [ ]:
df[df.columns[2:]].iloc[0]

In [ ]:
text = df.iloc[100]['comment_text']
prds = df[df.columns[2:]].iloc[100]
print(text)
print('----------')
print(prds)

In [ ]:
df[df['toxic']==1].head() # data is ok

# preprocess

In [ ]:
TextVectorization??

In [ ]:
df['comment_text']

In [ ]:
df.columns[2:].values

In [ ]:
df[df.columns[2:]].values

In [ ]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [ ]:
y

In [ ]:
max_words = 200000 #Number of our voab words :)))))

In [ ]:
type(X.values)

In [ ]:
vectorizer = TextVectorization(max_tokens=max_words, #vocab
                               output_sequence_length=1800, #the length of the sentences we are inputting
                               output_mode='int')

In [ ]:
vectorizer.adapt(X.values)

In [ ]:
vectorizer.get_vocabulary()

In [ ]:
vectorizer('Artin Daneshvar is a student')[:5] #Example of what we are doing

In [ ]:
vectorized_text = vectorizer(X.values)

In [ ]:
df.shape[0] #so it has 159571 sentences

In [ ]:
vectorized_text #each of those 159571 sentences are now a 1800 dimensional vec

In [ ]:
len(vectorized_text)

###Tensorflow data pipeline

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(159571)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [ ]:
dataset.as_numpy_iterator().next()

In [ ]:
train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

In [ ]:
len(dataset)

In [ ]:
len(train) + len(val) + len(test)

In [ ]:
### These are batches

In [ ]:
tr_generator = dataset.as_numpy_iterator()

In [ ]:
tr_generator.next()

# Create the model

In [ ]:
model = Sequential()
model.add(Embedding(max_words+1, 32))

model.add(Bidirectional(LSTM(32, activation='tanh')))

model.add(Dense(128, activation='relu'))

model.add(Dense(256, activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dense(6, activation='sigmoid'))

In [ ]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [ ]:
model.summary()

In [ ]:
history = model.fit(train, epochs=1, validation_data=val)

In [ ]:
history.history

In [ ]:
input_text = vectorizer('I KILL YOU')

In [ ]:
input_text

In [ ]:
batch = test.as_numpy_iterator().next()

In [ ]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [ ]:
(model.predict(batch_X) > 0.5).astype(int)

In [ ]:
res = (model.predict(np.expand_dims(input_text,0)) > 0.5).astype(int)

In [ ]:
print(res)

In [ ]:
model.save('toxicity.h5')

# Evaluate Model

In [ ]:
pre = Precision()
rec = Recall()
accu = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X_true, y_true = batch
    yhat = model.predict(X_true)

    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    rec.update_state(y_true, yhat)
    accu.update_state(y_true, yhat)

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall:{rec.result().numpy()}, Accuracy:{accu.result().numpy()}')

# Testing the model

In [ ]:
model = tf.keras.models.load_model('toxicity.h5')

In [ ]:
input_str = vectorizer('hey i freaken hate you!')

In [ ]:
res = model.predict(np.expand_dims(input_str,0))

In [ ]:
res

In [ ]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)

    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)

    return text

In [ ]:
score_comment('Killing people')